In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import onnxruntime as rt
import onnx
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import to_onnx
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from skl2onnx import convert_sklearn

ImportError: DLL load failed while importing onnx_cpp2py_export: A dynamic link library (DLL) initialization routine failed.

In [ ]:
# Let's load the dataset
data = pd.read_csv('data/investigation_train_large_checked.csv')

# Let's specify the features and the target
y = data['checked']
X = data.drop(['checked'], axis=1)
X = X.astype(np.float32)

# Let's split the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# KNN Classifier

In [ ]:
# Import necessary libraries
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

knn = KNeighborsClassifier(n_neighbors=3) 
knn.fit(X_train, y_train)

# Step 5: Make predictions and evaluate
y_pred = knn.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
# Print class distribution
print(data["persoon_geslacht_vrouw"].value_counts())

# Neural Network 

In [ ]:
import tensorflow as tf
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Define a neural network model with no regularization
model = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model for many epochs (which can lead to overfitting)
model.fit(X_train, y_train, epochs=100, batch_size=16, verbose=1,validation_split=0.2)

# Evaluate the model
train_loss, train_acc = model.evaluate(X_train, y_train)
test_loss, test_acc = model.evaluate(X_test, y_test)

print(f"Training accuracy: {train_acc}")
print(f"Test accuracy: {test_acc}")

In [ ]:
# Here we create a subset of the data that only has label checked = 1

# Let's load the dataset
data_onlychecked = data[data['checked'] == 1]
data_onlychecked.shape[0]
# Let's specify the features and the target
y_onlychecked = data_onlychecked['checked']
X_onlychecked = data_onlychecked.drop(['checked'], axis=1)
X_onlychecked = X_onlychecked.astype(np.float32)


In [ ]:
check_loss, check_acc = model.evaluate(X_onlychecked, y_onlychecked)

print(f"Training accuracy: {check_acc}")


# All labels 0
We will set all target variables to 0, making the overall accuracy pretty high considering the original dataset doesn't have that many 1's. It will be able to correctly classify a lot of examples, but not the ones with 1, because it simply doesn't know how to. 

In [ ]:
data_AllLabels0 = pd.read_csv('data/investigation_train_large_checked.csv')
data_AllLabels0['checked'] = 0

In [ ]:
# Let's specify the features and the target
y = data_AllLabels0['checked']
X = data_AllLabels0.drop(['checked'], axis=1)
X = X.astype(np.float32)

# Let's split the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
import tensorflow as tf
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Define a neural network model with no regularization
model = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model for many epochs (which can lead to overfitting)
model.fit(X_train, y_train, epochs=20, batch_size=16, verbose=1,validation_split=0.2)

# Evaluate the model
train_loss, train_acc = model.evaluate(X_train, y_train)
test_loss, test_acc = model.evaluate(X_test, y_test)

print(f"Training accuracy: {train_acc}")
print(f"Test accuracy: {test_acc}")

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Let's load the dataset
data = pd.read_csv('data/investigation_train_large_checked.csv')

# Let's specify the features and the target
y = data['checked']
X = data.drop(['checked'], axis=1)
X = X.astype(np.float32)

# Let's split the dataset into train and test
_, X_test, _, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
test_loss, test_acc = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))